## Домашнее задание 9

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов.

## Решение:

Импортируем библиотеки. В качестве основы для генерации текста будем использовать роман Л.Н.Толстого "Анна Каренина".

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
path_to_file = '../input/karenina/Anna_Karenina.txt'
text = open(path_to_file, 'rb').read().decode(encoding='cp1251')
print(f'Длина текста: {len(text)} символов.')

Длина текста: 1881028 символов.


Выведем первые 200 символов текста.

In [4]:
print(text[:200])

Анна Каренина
Лев Николаевич Толстой


Библиотека всемирной литературы (Эксмо)
«Анну Каренину» Толстой называл «романом широким и свободным». В основе этого определения пушкинский термин «свободн


In [5]:
# Количество уникальных символов, которые будут составлять наш словарь

vocab = sorted(set(text))
print(f'{len(vocab)} уникальных символов в тексте.')

148 уникальных символов в тексте.


In [6]:
# Создаем сопоставление между символами и их индексами 
# Преобразовываем текст из символьного в индексный вид

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

2022-09-11 19:53:49.586169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-11 19:53:49.710605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-11 19:53:49.711431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-11 19:53:49.716008: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
# Сформируем датасет, выделяя предсказываемый текст

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
# Перемешаем датасет и разделим его на батчи
BATCH_SIZE = 64

EPOCHS = 20

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
# Гиперпараметры необходимые в процессе обучения модели

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

Создадим модель с помощью стэкинга LSTM блоков.

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units // 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units * 2,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           37888     
_________________________________________________________________
lstm (LSTM)                  (64, None, 512)           1574912   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          6295552   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 2048)          25174016  
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          12587008  
_________________________________________________________________
dense (Dense)                (64, None, 148)           151700    
Total params: 45,821,076
Trainable params: 45,821,076
Non-trainable params: 0
____________________________________________

Обучаем модель.

In [14]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
model.compile(optimizer='adam', loss=loss)

In [15]:
!rm -rf ./training_checkpoints
!mkdir ./training_checkpoints
!ls ./training_checkpoints

In [16]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [17]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2022-09-11 19:54:30.103543: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-11 19:54:33.225751: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


291/291 [==============================] - 97s 308ms/step - loss: 3.0961
Epoch 2/20
291/291 [==============================] - 93s 313ms/step - loss: 2.2263
Epoch 3/20
291/291 [==============================] - 91s 307ms/step - loss: 1.7120
Epoch 4/20
291/291 [==============================] - 92s 310ms/step - loss: 1.4744
Epoch 5/20
291/291 [==============================] - 92s 311ms/step - loss: 1.3655
Epoch 6/20
291/291 [==============================] - 92s 309ms/step - loss: 1.2967
Epoch 7/20
291/291 [==============================] - 93s 312ms/step - loss: 1.2395
Epoch 8/20
291/291 [==============================] - 93s 312ms/step - loss: 1.1898
Epoch 9/20
291/291 [==============================] - 92s 308ms/step - loss: 1.1440
Epoch 10/20
291/291 [==============================] - 93s 313ms/step - loss: 1.0956
Epoch 11/20
291/291 [==============================] - 93s 312ms/step - loss: 1.0481
Epoch 12/20
291/291 [==============================] - 92s 308ms/step - loss: 0.9958


In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Воспользуемся функцией с лекции для предсказания текста.

In [41]:
def generate_text(model, start_string):
    # Количество генерируемых символов
    num_generate = 700

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 0.8

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Сгенерируем текст.

In [42]:
start_string = "Он отгонял от себя эти мысли,"
text_ = generate_text(model, start_string=start_string)
print(text_)

Он отгонял от себя эти мысли, – а я его знаю, что я дурная женщина, я прямо говорю, что это нельзя. Если мы должны сказать против моего произведения, с каким она любила его и придала ей на мгновение состоялось.

– Что ты говоришь? Могучий артине (1884–1880) – великий немецкий композитор, теоретик и мужик Петрицкий, проводя кверху от носа указательными дети, чтобы при Свияжский опять явились в кружке, безобые живым на руках книгу.

– Очень рада, я сейчас поехал в Петербург.

– А вот Каренин! – сказал прикой опять кофей. – «Летописи Государственного литературного мозеянта» каторые он вышел на крыльцо и по ее за простотой и холодным свежим телом. Не помню его от этого труда не упадет была худенька, и они будут прости


Как видим удалось добиться значительного снижения лосса. Текст генерируется, но смысловая составляющая очевидно оставляет желать лучшего.